In [2]:
import os

In [3]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )
        return data_transformation_config



In [17]:
import pandas as pd
data = pd.read_csv("F:\zizo\MLOPS\datascienceproject\data\Temp_dataset.csv")
len(data)
data.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,TAVG,TAVG_ATTRIBUTES
0,EG000062463,"HURGUADA, EG",27.15,33.7167,16.0,2000-01-01,20.6,"H,,S"
1,EG000062463,"HURGUADA, EG",27.15,33.7167,16.0,2000-01-02,19.7,"H,,S"
2,EG000062463,"HURGUADA, EG",27.15,33.7167,16.0,2000-01-03,20.9,"H,,S"
3,EG000062463,"HURGUADA, EG",27.15,33.7167,16.0,2000-01-04,17.3,"H,,S"
4,EG000062463,"HURGUADA, EG",27.15,33.7167,16.0,2000-01-05,14.6,"H,,S"


In [18]:
data = data.drop(["STATION","TAVG_ATTRIBUTES","NAME"],axis= 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98446 entries, 0 to 98445
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LATITUDE   98446 non-null  float64
 1   LONGITUDE  98446 non-null  float64
 2   ELEVATION  98446 non-null  float64
 3   DATE       98446 non-null  object 
 4   TAVG       98446 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [19]:
data['DATE'] = pd.to_datetime(data['DATE'])
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.month
data['DAY'] = data['DATE'].dt.day
data['DAY_OF_WEEK'] = data['DATE'].dt.dayofweek
data['DAY_OF_YEAR'] = data['DATE'].dt.dayofyear

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98446 entries, 0 to 98445
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   LATITUDE     98446 non-null  float64       
 1   LONGITUDE    98446 non-null  float64       
 2   ELEVATION    98446 non-null  float64       
 3   DATE         98446 non-null  datetime64[ns]
 4   TAVG         98446 non-null  float64       
 5   YEAR         98446 non-null  int32         
 6   MONTH        98446 non-null  int32         
 7   DAY          98446 non-null  int32         
 8   DAY_OF_WEEK  98446 non-null  int32         
 9   DAY_OF_YEAR  98446 non-null  int32         
dtypes: datetime64[ns](1), float64(4), int32(5)
memory usage: 5.6 MB


In [26]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def Preprocessing(self):
        data = pd.read_csv(self.config.data_path)
        data = data.drop(["STATION","TAVG_ATTRIBUTES","NAME"],axis= 1)

        data['DATE'] = pd.to_datetime(data['DATE'])
        data['YEAR'] = data['DATE'].dt.year
        data['MONTH'] = data['DATE'].dt.month
        data['DAY'] = data['DATE'].dt.day
        data['DAY_OF_WEEK'] = data['DATE'].dt.dayofweek
        data['DAY_OF_YEAR'] = data['DATE'].dt.dayofyear
                
        return data


        
    def train_test_splitting(self, data: pd.DataFrame):
        train, test = train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


In [28]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    preprocessing_data = data_transformation.Preprocessing()
    data_transformation.train_test_splitting(preprocessing_data)
except Exception as e:
    raise e

[2025-03-11 23:55:04,032: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-11 23:55:04,034: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-11 23:55:04,037: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-11 23:55:04,038: INFO: common: created directory at: artifacts]
[2025-03-11 23:55:04,039: INFO: common: created directory at: artifacts/data_transformation]
[2025-03-11 23:55:04,470: INFO: 4062491614: Splited data into training and test sets]
[2025-03-11 23:55:04,471: INFO: 4062491614: (73834, 10)]
[2025-03-11 23:55:04,472: INFO: 4062491614: (24612, 10)]
(73834, 10)
(24612, 10)
